# Multi-body Tutorial
Multi-body problems are polynomials where the total degree of at least one term in the polynomial is higher than 2. For instance
$$
f(x)=2x^3-x^2-x
$$
and 
$$
f(x,y)=xy^2-x^2y+x+y
$$ 
are two examples of cubic "multi-body" problems.

Multi-body support was added to `qci-client` in version 3.2.0. In order to utilize this tutorial, a version of `qci-client` of at least 4.0 is required.

In [ ]:
!pip install "qci-client>=4.0"

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import qci_client as qc
import eqc_direct.utils
import eqc_direct.client


In [2]:
client = qc.QciClient()

## Polynomial Format
The ability to support higher order terms efficiently for the majority of problems requires a sparse format. That format is inspired by polynomials themselves. Two different arrays are required. The first is a coefficient array. Each term in the function has an entry in the coefficient array. The second array, the indices array, is where to indicate which term the corresponding coefficient is for. The second polynomial from above is presented here.

$$
f(x,y)=xy^2-x^2y+x+y
$$ 
is represented in polynomial format
```
poly_coefficients = [1, -1, 1, 1]
poly_indices = [[1, 2, 2], [1, 1, 2], [0, 0, 1], [0, 0, 2]]
```

In [3]:
poly_coefficients = [1, -1, 1, 1]
poly_indices = [[1, 2, 2], [1, 1, 2], [0, 0, 1], [0, 0, 2]]
data = []
for i in range(len(poly_coefficients)):
    data.append({
        "val": poly_coefficients[i],
        "idx": poly_indices[i]
    })
poly_file = {"file_name": "test-polynomial",
             "file_config": {"polynomial": {
                 "min_degree": 1,
                 "max_degree": 3,
                 "num_variables": 2,
                 "data": data
             }}}
file_id = client.upload_file(file=poly_file)["file_id"]
file_id

'6629100598263204a3650319'

In [4]:
poly_file

{'file_name': 'test-polynomial',
 'file_config': {'polynomial': {'min_degree': 1,
   'max_degree': 3,
   'num_variables': 2,
   'data': [{'val': 1, 'idx': [1, 2, 2]},
    {'val': -1, 'idx': [1, 1, 2]},
    {'val': 1, 'idx': [0, 0, 1]},
    {'val': 1, 'idx': [0, 0, 2]}]}}}

In [5]:
job_body = client.build_job_body(job_type="sample-hamiltonian", polynomial_file_id=file_id, job_params={"device_type": "dirac-3", "sum_constraint": 10, "solution_precision": 1, "relaxation_schedule": 3, "num_samples": 5})
job_body

{'job_submission': {'problem_config': {'normalized_qudit_hamiltonian_optimization': {'polynomial_file_id': '6629100598263204a3650319'}},
  'device_config': {'dirac-3': {'num_samples': 5,
    'relaxation_schedule': 3,
    'solution_precision': 1,
    'sum_constraint': 10}}}}

In [6]:
response = client.process_job(job_body=job_body)

2024-04-24 07:58:30 - Dirac allocation balance = 0 s (unmetered)
2024-04-24 07:58:30 - Job submitted: job_id='66291006bdefceebf853c906'
2024-04-24 07:58:30 - QUEUED
2024-04-24 07:58:32 - RUNNING
2024-04-24 07:59:54 - COMPLETED
2024-04-24 07:59:57 - Dirac allocation balance = 0 s (unmetered)


In [7]:
response

{'job_info': {'job_id': '66291006bdefceebf853c906',
  'job_submission': {'problem_config': {'normalized_qudit_hamiltonian_optimization': {'polynomial_file_id': '6629100598263204a3650319'}},
   'device_config': {'dirac-3': {'num_samples': 5,
     'relaxation_schedule': 3,
     'solution_precision': 1,
     'sum_constraint': 10}}},
  'job_status': {'submitted_at_rfc3339nano': '2024-04-24T13:58:30.292Z',
   'queued_at_rfc3339nano': '2024-04-24T13:58:30.292Z',
   'running_at_rfc3339nano': '2024-04-24T13:58:31.09Z',
   'completed_at_rfc3339nano': '2024-04-24T13:59:53.093Z'},
  'job_result': {'file_id': '6629105998263204a365031b', 'device_usage_s': 10}},
 'status': 'COMPLETED',
 'results': {'counts': [2, 1, 1, 1],
  'energies': [-85.8416367, -86.2230377, -86.172554, -86.0685577],
  'solutions': [[8.0342817, 1.965718],
   [7.8760204, 2.1239791],
   [7.9416203, 2.0583792],
   [7.9812946, 2.0187058]],
  'distilled_energies': [-86, -86, -86, -86],
  'distilled_solutions': [[8, 2], [8, 2], [8, 2]